# Unit Test Notebook for Gripper class functions

In [ ]:
import sys
sys.path.append('../../')
from magpie.gripper import Gripper
import time
servoport = '/dev/ttyACM0'
G = Gripper(servoport=servoport)

In [ ]:
# Open the gripper to set aperture
theta = G.aperture_to_theta(10) # 10mm aperture between finger and camera x-center
G.Finger1 = G.set_goal_position(G.theta_to_position(theta, finger='left'))
G.Finger2 = G.set_goal_position(G.theta_to_position(theta, finger='right'))
# validate z-offset (10mm aperture on either finger should be 20mm)
print(f"z-offset: {G.aperture_to_z(20, finger='both')} mm")

In [ ]:
# Validate the same distance
G.set_goal_aperture(20, finger='both') # 10mm aperture between each finger and camera x-center, totalling 20mm
# validate z-offset
print(f"z-offset: {G.aperture_to_z(20, finger='both')} mm")

In [ ]:
# Compare with Dylan's code
G.set_goal_distance(20) # 10mm aperture between each finger and camera x-center, totalling 20mm 

# Testing Getters and Setters

## Getters

In [ ]:
print(G.get_load(finger='both'))

In [ ]:
print(G.get_position(finger='both'))

In [ ]:
print(G.get_temperature(finger='both'))

In [ ]:
print(G.get_aperture(finger='both'))

In [ ]:
# compare with Dylan's code
print(G.get_distance(finger='both'))

## Setters

In [ ]:
# 1 N
G.set_force(1.0, finger='both', debug=True)
time.sleep(0.005)
G.get_load(finger='both')

In [ ]:
# set distance to 20 mm 
G.set_goal_distance(20, debug=True)
time.sleep(0.5)
print(G.get_distance(finger='both'))
print(G.get_aperture(finger='both'))
print(G.get_position(finger='both'))

In [ ]:
# set compliance (in mm, slope[0-7])
G.set_compliance(5, 4, finger='both', debug=True)
time.sleep(0.005) 
print(G.get_compliance(finger='both'))


In [2]:
# https://platform.openai.com/docs/guides/vision
import openai
import base64
import requests
import os
from PIL import Image
from pillow_heif import register_heif_opener
import torch
from transformers.image_utils import ImageFeatureExtractionMixin

# import open3d as o3d
import numpy as np
# import pyrealsense2 as rs
import matplotlib.pyplot as plt
# import RealSense as real

# # Initialize RS435i connection
# rsc = real.RealSense()
# rsc.initConnection()


In [25]:
api_key = os.getenv("OPENAI_API_KEY")


In [31]:
# Path to your image
image_path = "data_snapshots/rgb_image_ur5.png"
image_path_labeled = 'data_snapshots/owlvit_inference.png'

# Getting the base64 string
base64_image = encode_image(image_path)
base64_image_labeled = encode_image(image_path_labeled)

api_key = os.getenv("OPENAI_API_KEY")

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

text1 = """
You are planning a grasp for a robotic manipulator in a grocery bin. 
You will validate whether the labeled objects are present in the bin and correct the list of labels if there are any incongruities.
For example if a the label in the image is "apple (3)" but the object is a "pear", you will correct the label to "pear (3)" in the format of the provided caption.
The number in the image label corresponds to the index of the object in the list of objects.
You are provided with a photo of the grocery bin, a photo of the indexed and labeled grocery bins, and a list of labels with matching indices.
In your response, first identify any errors in the labels or missing labels. Then, finish your response in the format of the provided caption, with corrections applied.
Make sure to mention the index of the fruit associated with the error.
"""

resp1 = """
I see: ["a photo of a lemon", "a photo of an apple", "a photo of a pear", "a photo of an onion", "a photo of a lime"]
Labels: ["lemon", "apple", "pear", "onion", "lime"]
"""

text2 = """
Please generate a list of labels for this new image of the grocery bin, in the same format.
Are there any new fruits or vegetables?
"""
question = """
No further information required.
Now I am looking at a painting of people looking at a shore.
"""

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{text1}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image_labeled}"
            }
          }]
      },
      # {
      #   "role": "assistant",
      #   "content": [
      #     {
      #       "type": "text",
      #       "text": f"{resp1}"
      #     }]
      # },
      # {
      #   "role": "user",
      #   "content": [
      #     {
      #       "type": "text",
      #       "text": f"{text2}"
      #     },
      #     {
      #       "type": "image_url",
      #       "image_url": {
      #         "url": f"data:image/jpeg;base64,{base64_image_avo}"
      #       }
      #     }]
      # }          
      ],
    "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8q2WxRjlJrltz2XHTlAbPBmXc1HHz', 'object': 'chat.completion', 'created': 1707413475, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 2058, 'completion_tokens': 169, 'total_tokens': 2227}, 'choices': [{'message': {'role': 'assistant', 'content': 'Based on the provided images, the following errors in the labels have been identified:\n\n1. The object labeled as "orange (4)" is actually a red apple.\n2. The item labeled as "orange (5)" is a red apple.\n3. The item labeled as "orange (9)" is a green pear.\n\nGiven this information, the corrected list of labels with matching indices is:\n\n- orange (0)\n- orange (1)\n- orange (2)\n- apple (3)\n- apple (4)\n- orange (6)\n- orange (7)\n- orange (8)\n- pear (9)\n- orange (10)\n- orange (11)\n- orange (12)\n- orange (13)\n- orange (14)\n- orange (15)\n- orange (16)\n- orange (17)\n- orange (18)'}, 'finish_reason': 'stop', 'index': 0}]}


In [32]:
generation = response.json()["choices"][0]['message']['content']
generation

'Based on the provided images, the following errors in the labels have been identified:\n\n1. The object labeled as "orange (4)" is actually a red apple.\n2. The item labeled as "orange (5)" is a red apple.\n3. The item labeled as "orange (9)" is a green pear.\n\nGiven this information, the corrected list of labels with matching indices is:\n\n- orange (0)\n- orange (1)\n- orange (2)\n- apple (3)\n- apple (4)\n- orange (6)\n- orange (7)\n- orange (8)\n- pear (9)\n- orange (10)\n- orange (11)\n- orange (12)\n- orange (13)\n- orange (14)\n- orange (15)\n- orange (16)\n- orange (17)\n- orange (18)'

In [40]:
resp1 = """
Here is an example of a correct response to these images:
Based on the provided images, the following errors in the labels have been identified:
n1. The object labeled as "orange (0)" in the top right is actually a yellow lemon.
n2. The item labeled as "orange (3)" to the left of "orange (0)" is a yellow lemon.
n3. The item labeled as "orange (6)" in the far right is a yellow lemon.
n2. The item labeled as "orange (8)" to the left of "orange (6)" is a yellow lemon.
Given this information, the corrected list of labels with matching indices is:\n\n- lemon (0)\n- orange (1)\n- orange (2)\n- lemon (3)\n- orange (4)\n- orange (5)\n- lemon (6)\n- orange (7)\n- lemon (8)\n- orange (9)\n- orange (10)\n- orange (11)\n- orange (12)\n- orange (13)\n- orange (14)\n- orange (15)\n- orange (16)\n- orange (17)\n- orange (18)
"""

In [41]:
text1 = """
You are planning a grasp for a robotic manipulator in a grocery bin. 
You will validate whether the labeled objects are present in the bin and correct the list of labels if there are any incongruities.
For example if a the label in the image is "apple (3)" but the object is a "pear", you will correct the label to "pear (3)" in the format of the provided caption.
The number in the image label corresponds to the index of the object in the list of objects.
You are provided with a photo of the grocery bin, a photo of the indexed and labeled grocery bins, and a list of labels with matching indices.
In your response, first identify any errors in the labels or missing labels. Then, finish your response in the format of the provided caption, with corrections applied.
Make sure to mention the index of the fruit associated with the error.

Here is an example of a caption:
"""
text1 = text1 + context

In [45]:
text2 = """Now, please identifiy any errors or missing labels in the next image. Provided caption: """
text2 = text2 + context_pear
text2

"Now, please identifiy any errors or missing labels in the next image. Provided caption: \npear (0): ([1003.6107110977173, 109.06752079725266, 1086.157660484314, 173.9412572979927], 'pear')\npear (1): ([823.7149906158447, 91.88447713851929, 982.2424983978271, 239.59284782409668], 'pear')\npear (2): ([573.7592506408691, 218.29054534435272, 660.1446342468262, 311.40706837177277], 'pear')\npear (3): ([620.7485294342041, 263.33383083343506, 804.0051937103271, 411.74015522003174], 'pear')\npear (4): ([953.1515741348267, 304.79566991329193, 1039.001441001892, 410.0387817621231], 'pear')\npear (5): ([624.2333507537842, 260.7258278131485, 803.745927810669, 409.4582122564316], 'pear')\npear (6): ([948.1863212585449, 302.46907353401184, 1031.769733428955, 430.90051531791687], 'pear')\npear (7): ([501.7671251296997, 371.20080292224884, 600.0220441818237, 477.94571578502655], 'pear')\npear (8): ([579.2618751525879, 393.62665593624115, 659.7358512878418, 501.1235100030899], 'pear')\n"

In [50]:
# Path to your image
image_path = "data_snapshots/rgb_image_ur5.png"
image_path_orange = 'data_snapshots/owlvit_inference-orange.png'
image_path_pear = 'data_snapshots/owlvit_inference-pear.png'

# Getting the base64 string
base64_image = encode_image(image_path)
base64_image_orange = encode_image(image_path_orange)
base64_image_pear = encode_image(image_path_pear)

api_key = os.getenv("OPENAI_API_KEY")

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}



# resp1 = """
# I see: ["a photo of a lemon", "a photo of an apple", "a photo of a pear", "a photo of an onion", "a photo of a lime"]
# Labels: ["lemon", "apple", "pear", "onion", "lime"]
# """

text2 = """
Please generate a list of labels for this new image of the grocery bin, in the same format.
Are there any new fruits or vegetables?
"""
question = """
No further information required.
Now I am looking at a painting of people looking at a shore.
"""

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{text1}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image_orange}"
            }
          }
          ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": f"{resp1}"
          }]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{text2}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
          ,
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image_pear}"
            }
          }
          ]
      }          
      ],
    "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8q2phOt1556w4tx7WqU92SYBQLnYW', 'object': 'chat.completion', 'created': 1707414637, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 3922, 'completion_tokens': 104, 'total_tokens': 4026}, 'choices': [{'message': {'role': 'assistant', 'content': 'Based on the provided image, here is a list of labels with corrections to the indices and types of fruits they correspond to:\n\n- pear (0)\n- pear (1)\n- pear (2)\n- pear (3)\n- pear (4)\n- pear (5)\n- pear (6)\n- pear (7)\n- pear (8)\n\nThere are no new fruits or vegetables in the image beyond the pears. Each labeled item appears to be a pear, correctly matching the label. No adjustments are needed.'}, 'finish_reason': 'stop', 'index': 0}]}


In [51]:
generation = response.json()["choices"][0]['message']['content']
generation

'Based on the provided image, here is a list of labels with corrections to the indices and types of fruits they correspond to:\n\n- pear (0)\n- pear (1)\n- pear (2)\n- pear (3)\n- pear (4)\n- pear (5)\n- pear (6)\n- pear (7)\n- pear (8)\n\nThere are no new fruits or vegetables in the image beyond the pears. Each labeled item appears to be a pear, correctly matching the label. No adjustments are needed.'